In [132]:
import pandas as pd
import numpy as np
import glob
import os
import csv
import sys
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from scipy.stats import linregress
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"
import plotly.express as px


#sns.set_style('darkgrid')

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib

#setting pandas display options
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

BASE_DIR = "/Users/karinalopez/Desktop/ds_projects/hack4la_311/"

<ipython-input-132-212a6b153825>:30: FutureWarning:

Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.



In [93]:
os.chdir(BASE_DIR + 'data/intermediate/')
df = pd.read_csv('clean_311data.csv')

In [94]:
df.head()

,index,location,zipcode,suffix,srnumber,updateddate,closeddate,cd,address,createdbyuserorganization,createddate,requestsource,requesttype,assignto,latitude,owner,housenumber,mobileos,approximateaddress,policeprecinct,cdmember,servicedate,nc,longitude,tbmcolumn,status,tbmrow,direction,anonymous,addressverified,actiontaken,streetname,ncname,apc,tbmpage,days_to_service,days_to_close,days_to_update
0,0,"{'latitude': '33.9589038232', 'needs_recoding': False, 'longitude': '-118.273259328'}",90003,ST,1-1831739761,2021-01-02 08:12:27,2021-01-02 07:39:24,8,"114 E 87TH ST, 90003",Self Service,2020-12-31 23:57:54,Mobile App,Bulky Items,SLA,33.958904,LASAN,114.0,Android,NaN,SOUTHEAST,Marqueece Harris-Dawson,2021-01-02 00:00:00,87.0,-118.273259,C,Closed,3.0,E,N,Y,SR Created,87TH,Empowerment Congress Southeast,South Los Angeles APC,704.0,1.0,1.0,1.0
1,2,"{'latitude': '34.1794105977', 'needs_recoding': False, 'longitude': '-118.373220799'}",91601,NaN,1-1831738771,2021-01-04 15:49:43,2021-01-04 15:49:43,2,"FULCHER AVE AT OXNARD ST, 91601",Self Service,2020-12-31 23:54:33,Self Service,Graffiti Removal,NDFY,34.179411,OCB,NaN,NaN,N,NORTH HOLLYWOOD,Paul Krekorian,2021-01-04 00:00:00,24.0,-118.373221,J,Cancelled,7.0,NaN,Y,Y,SR Created,NaN,NoHo,South Valley APC,532.0,3.0,3.0,3.0
2,4,"{'latitude': '34.1868286824', 'needs_recoding': False, 'longitude': '-118.391412973'}",91606,BLVD,1-1831736811,2021-01-05 15:28:44,2021-01-05 15:26:58,2,"11841 W VICTORY BLVD, 91606",Self Service,2020-12-31 23:48:56,Self Service,Bulky Items,EV,34.186829,LASAN,11841.0,NaN,N,NORTH HOLLYWOOD,Paul Krekorian,2021-01-05 00:00:00,23.0,-118.391413,G,Closed,6.0,W,Y,Y,SR Created,VICTORY,North Hollywood Northeast,South Valley APC,532.0,4.0,4.0,4.0
3,5,"{'latitude': '34.1722631848', 'needs_recoding': False, 'longitude': '-118.544654655'}",91356,AVE,1-1831736791,2021-01-07 23:19:34,2021-01-07 19:02:27,3,"5600 N WILBUR AVE, 91356",LASAN,2020-12-31 23:48:47,Call,Homeless Encampment,WV,34.172263,LASAN,5600.0,NaN,N,WEST VALLEY,Bob Blumenfield,2021-01-07 00:00:00,17.0,-118.544655,H,Closed,2.0,N,N,Y,SR Created,WILBUR,Tarzana,South Valley APC,560.0,6.0,6.0,6.0
4,6,"{'latitude': '34.1865017598', 'needs_recoding': False, 'longitude': '-118.383920351'}",91606,BLVD,1-1831739561,2021-01-06 03:49:36,2021-01-05 08:53:40,2,"11542 W VICTORY BLVD, 91606",Self Service,2020-12-31 23:47:50,Self Service,Bulky Items,EV,34.186502,LASAN,11542.0,NaN,N,NORTH HOLLYWOOD,Paul Krekorian,2021-01-05 00:00:00,24.0,-118.383920,H,Closed,6.0,W,Y,Y,SR Created,VICTORY,NoHo,South Valley APC,532.0,4.0,4.0,5.0


In [95]:
df['ncname'].unique()

array(['Empowerment Congress Southeast', 'NoHo',
       'North Hollywood Northeast', 'Tarzana', 'Mar Vista', 'Eagle Rock',
       'CANNDU', 'LA32', 'Northwest San Pedro', 'Northridge West',
       'Panorama City', 'Coastal San Pedro', 'West Hills', 'Arleta',
       'Foothill Trails District', 'Empowerment Congress West',
       'Greater Wilshire', 'Lake Balboa', 'Historic Highland Park',
       'Reseda', 'Mid City West', 'Palms', 'North Hills East',
       'Studio City', 'South Robertson', 'Echo Park', 'Central San Pedro',
       'Granada Hills North', 'Olympic Park', 'Central Alameda',
       'Pacoima', 'Bel Air-Beverly Crest', 'Porter Ranch', 'Sherman Oaks',
       'Van Nuys', 'Mission Hills', 'Sylmar', 'Pico Union',
       'Woodland Hills-Warner Center', 'Hollywood Hills West',
       'Hollywood Studio District', 'Harbor Gateway North',
       'Boyle Heights', 'Canoga Park', 'Lincoln Heights',
       'Westchester/Playa', 'Greater Toluca Lake', 'P.I.C.O.',
       'Greater Valley Glen

In [96]:
df['ncname'].nunique()

203

In [97]:
df['cd'].nunique()

15

In [98]:
df['policeprecinct'].nunique()


21

**most recent council district boundary has been updated in 2012**

- District 1: Gil Cedillo X
- District 2: Paul Krekorian X
- District 3: Bob Blumenfield X
- District 4: Nithya Raman X
- District 5: Paul Koretz X
- District 6: Nury Martinez (President) X
- District 7: Monica Rodriguez X
- District 8: Marqueece Harris-Dawson X
- District 9: Curren Price X
- District 10: Mark Ridley-Thomas Los Angeles City Council Member since 2020 (NEWLY elected possibly to district 10)
- District 11: Mike Bonin X
- District 12: John Lee[1] X
- District 13: Mitch O'Farrell X
- District 14: Kevin De Leon X
- District 15: Joe Buscaino X

    
**NOT ON WIKIPEDIA:**
- Herb Wesson (former, distrct 10)
- Jose Huizar (former district member; left in 2020, arrested and indicted on corruption charges in 2020 for district 14)
- David Ryu (former district 4), replaced by Nithya Raman in 2020
- Greig Smith (former district 12, replaced by Mitch O'Farrell in 2019/2020)
- Mitchell Englander (former for district 12, resigned in 2018)

In [99]:
df['cdmember'].unique()

array(['Marqueece Harris-Dawson', 'Paul Krekorian', 'Bob Blumenfield',
       'Mike Bonin', 'Kevin de León', 'Curren D. Price Jr.',
       'Joe Buscaino', 'John Lee', 'Nury Martinez', 'Monica Rodriguez',
       'Mark Ridley-Thomas', 'Paul Koretz', 'Gilbert Cedillo',
       "Mitch O'Farrell", 'Nithya Raman', 'Jose Huizar',
       'Herb J. Wesson Jr.', 'David Ryu', 'Greig Smith',
       'Mitchell Englander'], dtype=object)

In [100]:
df.shape

(1145748, 38)

# preparing dataset for dahsboard widget

In [101]:
# change date to date time colum
df['createddate'] = pd.to_datetime(df['createddate'])
df['servicedate'] = pd.to_datetime(df['servicedate'])

In [102]:
# create month and year columns
df['created_year'] = pd.DatetimeIndex(df['createddate']).year
df['created_month'] = pd.DatetimeIndex(df['createddate']).month

df['serviced_year'] = pd.DatetimeIndex(df['servicedate']).year
df['serviced_month'] = pd.DatetimeIndex(df['servicedate']).month

In [103]:
# grouby createddate (month and year), cd, requesttype, count number of listings
grouped_df = df.groupby(['created_month', 'created_year', 'cd', 'requesttype'])['requesttype'].count().reset_index(name = 'count_service')


In [104]:
# get total of grouped services into a single 
grouped_df.head()

,created_month,created_year,cd,requesttype,count_service
0,1,2020,1,Bulky Items,3088
1,1,2020,1,Electronic Waste,203
2,1,2020,1,Graffiti Removal,1206
3,1,2020,1,Homeless Encampment,231
4,1,2020,1,Illegal Dumping Pickup,568


In [105]:
# count total services requested by month/year/cd
grouped_count_df = df.groupby(['created_month', 'created_year', 'cd'])['createddate'].count().reset_index(name = 'total_requests')

In [106]:
grouped_count_df.head()

,created_month,created_year,cd,total_requests
0,1,2020,1,5890
1,1,2020,2,6694
2,1,2020,3,6079
3,1,2020,4,5891
4,1,2020,5,4826


In [107]:
# grouby createddate (month and year), cd, requesttype, count number of listings
grouped_average_date_df = df.groupby(['created_month', 'created_year', 'cd', 'requesttype'])['days_to_close'].mean().reset_index(name = 'avg_request_closed')

grouped_average_date_df['avg_request_closed'] = grouped_average_date_df['avg_request_closed'].round().astype(int)
grouped_average_date_df.head()


,created_month,created_year,cd,requesttype,avg_request_closed
0,1,2020,1,Bulky Items,4
1,1,2020,1,Electronic Waste,3
2,1,2020,1,Graffiti Removal,4
3,1,2020,1,Homeless Encampment,56
4,1,2020,1,Illegal Dumping Pickup,11


In [108]:
# rename months
month_dict = {1: 'January', 
              2: 'February', 
              3: 'March',
              4: 'April', 
              5: 'May', 
              6: 'June', 
              7: 'July', 
              8: 'August',
              9: 'September', 
              10: 'October',
              11: 'November', 
              12: 'December'}


In [109]:
# map new values to months
grouped_df['created_month'] = grouped_df['created_month'].replace(month_dict)
grouped_df['serviced_month'] = grouped_df['created_month'].replace(month_dict)

grouped_count_df['created_month'] = grouped_count_df['created_month'].replace(month_dict)
grouped_count_df['serviced_month'] = grouped_count_df['created_month'].replace(month_dict)

grouped_average_date_df['created_month'] = grouped_average_date_df['created_month'].replace(month_dict)
grouped_average_date_df['serviced_month'] = grouped_average_date_df['created_month'].replace(month_dict)


In [111]:
# merge both dataframes together
final_df = pd.merge(grouped_count_df, grouped_df, how = 'outer')
final_df = pd.merge(final_df, grouped_average_date_df, how = 'outer')

In [112]:
# calculate percentage
final_df['percentage_service'] = round((final_df['count_service']/final_df['total_requests']) * 100, 2)


In [113]:
# rename council districts
final_df['cd'] = 'Council District ' + final_df['cd'].astype(str)


In [114]:
final_df.head()

,created_month,created_year,cd,total_requests,serviced_month,requesttype,count_service,avg_request_closed,percentage_service
0,January,2020,Council District 1,5890,January,Bulky Items,3088,4,52.43
1,January,2020,Council District 1,5890,January,Electronic Waste,203,3,3.45
2,January,2020,Council District 1,5890,January,Graffiti Removal,1206,4,20.48
3,January,2020,Council District 1,5890,January,Homeless Encampment,231,56,3.92
4,January,2020,Council District 1,5890,January,Illegal Dumping Pickup,568,11,9.64


In [115]:
# create csv
os.chdir(BASE_DIR + 'scripts/interactive/')
final_df.to_csv('distribution_requests_by_date_cd.csv', index = False)

# create plotly plot

In [116]:
# create your mask filter
temp = final_df.loc[final_df['cd'] == 'Council District 1']
    
# filter by month and year as well
temp = temp.loc[final_df['created_month'] == 'January']
temp = temp.loc[final_df['created_year'] == 2020]


In [117]:
temp.head()

,created_month,created_year,cd,total_requests,serviced_month,requesttype,count_service,avg_request_closed,percentage_service
0,January,2020,Council District 1,5890,January,Bulky Items,3088,4,52.43
1,January,2020,Council District 1,5890,January,Electronic Waste,203,3,3.45
2,January,2020,Council District 1,5890,January,Graffiti Removal,1206,4,20.48
3,January,2020,Council District 1,5890,January,Homeless Encampment,231,56,3.92
4,January,2020,Council District 1,5890,January,Illegal Dumping Pickup,568,11,9.64


In [136]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np

os.chdir(BASE_DIR + 'scripts/interactive/')
final_df.to_csv('distribution_requests_by_date_cd.csv', index = False)


# Creating two subplots
fig = make_subplots(rows = 1, cols = 2, specs = [[{}, {}]], shared_xaxes = True,
                    shared_yaxes = False, vertical_spacing = 0.001)

# create first barplot
fig.append_trace(go.Bar(
    x = temp['count_service'],
    y = temp['requesttype'],
    marker = dict(
        color = 'rgba(50, 171, 96, 0.6)',
        line = dict(
            color = 'rgba(50, 171, 96, 1.0)',
            width = 1),
    ),
    
    name = 'Number of service requests per month',
    orientation = 'h',
), 1, 1)

# create second barplot
fig.append_trace(go.Bar(
   
    x = temp['avg_request_closed'], 
    y = temp['requesttype'],
    marker = dict(
        color = 'rgba(128, 0, 128, 0.6)',
        line = dict(
            color = 'rgba(128, 0, 128, 1.0)',
            width = 1),
    ),
    
    name = 'Average number of days to close request',
    orientation = 'h',
), 1, 2)



# update extra text
annotations = []


# Source
annotations.append(dict(xref = 'paper', yref = 'paper',
                        x = -0.2, y = -0.109,
                        text = ' WHEREVER WE GOT THIS DATASET FROM...',
                        font = dict(family = 'Arial', size = 10, color = 'rgb(150, 150, 150)'),
                        showarrow = False))

fig.update_layout(annotations = annotations)

fig.show()

#neighborhood council, are they failing to take care of my problems over another 
# what are the problems in my neighborhood council?

# e.g., cases of graffiti that. are not getting fixed in over a year




In [144]:
import plotly.express as px
 
fig = px.bar(x = temp['count_service'], color = temp['requesttype'], y = temp['cd'], barmode = 'stack')
 
fig.show()